In [ ]:
import music21
import pathlib

from thesession.converter import ABCMusicConverter

In [ ]:
ABCMusicConverter("cooleys.abc", "cooleys").to_mp3(
    instrument="flute",
    tempo=190,
    max_notes=300,
    cut_silence=30,
    start=0.57,
    noise_amplitude=0.001,
    duration=120,
    vbr=8,
    clean_files=True
)

# CLAP

In [ ]:
import torch
from laion_clap import CLAP_Module

# Load pretrained CLAP model
device = "cuda" if torch.cuda.is_available() else "cpu"
model = CLAP_Module(enable_fusion=True) # For using fusion to manage >10s clips
model.load_ckpt()  # downloads pretrained weights
model = model.to(device)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [11]:
import torchaudio

waveform, sr = torchaudio.load("cooleys.mp3")

In [12]:
import torchaudio
import torch


def load_audio(filepath, target_sr=16000):
    waveform, sr = torchaudio.load(filepath)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, sr, target_sr)
    return waveform.mean(dim=0).unsqueeze(0)  # Convert to mono, add batch dim

In [13]:
audio_tensor0 = load_audio("audio/1_cooleys/36371_0.mp3")
audio_tensor1 = load_audio("audio/1_cooleys/36371_1.mp3")
audio_tensor2 = load_audio("audio/8_the_banshee/32846_0.mp3")
audio_tensor3 = load_audio("audio/8_the_banshee/32846_1.mp3")

In [14]:
with torch.no_grad():
    embedding0 = model.get_audio_embedding_from_data(audio_tensor0, use_tensor=True)
    embedding1 = model.get_audio_embedding_from_data(audio_tensor1, use_tensor=True)
    embedding2 = model.get_audio_embedding_from_data(audio_tensor2, use_tensor=True)
    embedding3 = model.get_audio_embedding_from_data(audio_tensor3, use_tensor=True)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


In [15]:
embeddings = torch.cat([embedding0, embedding1, embedding2, embedding3], dim=0)
embeddings.shape

torch.Size([4, 512])

In [16]:
import torch.nn.functional as F

F.cosine_similarity(embeddings[None, :, :], embeddings[:, None, :], dim=-1)

tensor([[1.0000, 0.8502, 0.8558, 0.8473],
        [0.8502, 1.0000, 0.9414, 0.8412],
        [0.8558, 0.9414, 1.0000, 0.8524],
        [0.8473, 0.8412, 0.8524, 1.0000]], device='cuda:0')

# Dataset

In [1]:
import pathlib
import numpy as np
import pandas as pd

from thesession.dataset import TheSessionDataset

path = pathlib.Path("audio_flac")
tunes = [p.stem for p in path.iterdir() if p.is_dir()]
tunes.sort()

tunes_df = pd.DataFrame({"tune": tunes.copy(), "dataset": None})

prng = np.random.default_rng(42)
tunes = prng.permutation(tunes)

tunes_df.loc[tunes_df["tune"].isin(tunes[0 : int(0.15 * len(tunes))]), "dataset"] = "test"
tunes_df.loc[tunes_df["tune"].isin(tunes[int(0.15 * len(tunes)) : int(0.3 * len(tunes))]), "dataset"] = "validation"
tunes_df.loc[tunes_df["tune"].isin(tunes[int(0.3 * len(tunes)) :]), "dataset"] = "train"


tunes_df.to_csv("dataset.csv")

In [2]:
test_subset = tunes_df.loc[tunes_df["dataset"] == "test", "tune"]
val_subset = tunes_df.loc[tunes_df["dataset"] == "validation", "tune"]
train_subset = tunes_df.loc[tunes_df["dataset"] == "train", "tune"]

train_dataset = TheSessionDataset("audio_flac", subset=train_subset, sampling_rate=48000, fmt=".flac")
val_dataset = TheSessionDataset("audio_flac", subset=val_subset, sampling_rate=48000, fmt=".flac")
test_dataset = TheSessionDataset("audio_flac", subset=test_subset, sampling_rate=48000, fmt=".flac")

print("Training data", len(train_dataset))
print("Validation data", len(val_dataset))
print("Test data", len(test_dataset))

Training data 26344
Validation data 5534
Test data 5728


In [3]:
train_dataset[32]

(tensor([-0.0138, -0.0157, -0.0151,  ..., -0.0132, -0.0090, -0.0040]),
 tensor([ 0.0086,  0.0108,  0.0123,  ..., -0.0052, -0.0035, -0.0016]))

In [4]:
train_dataset[32][0].max()

tensor(0.0609)

In [5]:
train_dataset[32][0].min()

tensor(-0.0386)

In [6]:
%%timeit

train_dataset[0]

47 ms ± 3.97 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [7]:
import torch

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4)

In [8]:
import time

start = time.perf_counter()

for i, (x1, x2) in enumerate(train_loader):
    if i < 20:
        print(i, end="\r")
    else:
        break

end = time.perf_counter()
print("Duration: ", end - start)



Duration:  8.030320544996357


# Model

In [1]:
from thesession.model import TheSessionModel

model = TheSessionModel()
model.load_ckpt()

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

In [2]:
model.toggle_gradients(False)

Disabling gradient for parameter clap_model.model.logit_scale_a
Disabling gradient for parameter clap_model.model.logit_scale_t
Disabling gradient for parameter clap_model.model.audio_branch.spectrogram_extractor.stft.conv_real.weight
Disabling gradient for parameter clap_model.model.audio_branch.spectrogram_extractor.stft.conv_imag.weight
Disabling gradient for parameter clap_model.model.audio_branch.logmel_extractor.melW
Disabling gradient for parameter clap_model.model.audio_branch.bn0.weight
Disabling gradient for parameter clap_model.model.audio_branch.bn0.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.proj.weight
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.proj.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.norm.weight
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.norm.bias
Disabling gradient for parameter clap_model.model.audio_branch.patch_embed.mel_conv

In [3]:
model.toggle_gradients(True, ['clap_model.model.audio_projection'])

Enabling gradient for parameter clap_model.model.audio_projection.0.weight
Enabling gradient for parameter clap_model.model.audio_projection.0.bias
Enabling gradient for parameter clap_model.model.audio_projection.2.weight
Enabling gradient for parameter clap_model.model.audio_projection.2.bias


# Entraînement

In [11]:
import pandas as pd

from thesession.dataset import TheSessionDataset
from thesession.model import TheSessionModel
from thesession.training import eval_model, NTXentLoss

# Dataset
dataset = pd.read_csv("dataset.csv")

test_subset = dataset.loc[dataset["dataset"] == "test", "tune"]

test_dataset = TheSessionDataset(
    "audio_flac", subset=test_subset, sampling_rate=48000, fmt=".flac"
)

model = TheSessionModel(device="cuda")
criterion = NTXentLoss(temperature=0.05)
model.eval()

with open("testing.csv", "w") as f:
    f.write("\nstep,test_loss,test_accuracy")


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [12]:
# Baseline
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nbaseline,{loss},{acc}")

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


Duration of evaluation on test data: 79.15 seconds
Test Loss: 4.1314 | Test Accuracy: 0.0402 | 


In [13]:
# Pretrained weights
model.load_ckpt()
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\npretrain,{loss},{acc}")

Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.patch_embed.mel_conv2d.weight 	 Loaded
audio_branch.patch_embed.mel_conv2d.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.0.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.weight 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.1.bias 	 Loaded
audio_branch.patch_embed.fusion_model.local_att.3.weight 	 Loaded
audio_branc

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


Duration of evaluation on test data: 78.15 seconds
Test Loss: 5.5868 | Test Accuracy: 0.0102 | 


In [17]:
model.load("models/step1_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep1,{loss},{acc}")


Duration of evaluation on test data: 76.79 seconds
Test Loss: 1.3755 | Test Accuracy: 0.8455 | 


In [18]:
model.load("models/step2_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep2,{loss},{acc}")

Duration of evaluation on test data: 77.31 seconds
Test Loss: 1.2046 | Test Accuracy: 0.8943 | 


In [19]:
model.load("models/step3_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep3,{loss},{acc}")

Duration of evaluation on test data: 78.79 seconds
Test Loss: 0.3282 | Test Accuracy: 0.9010 | 


In [20]:
model.load("models/step4_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep4,{loss},{acc}")

Duration of evaluation on test data: 78.08 seconds
Test Loss: 0.1179 | Test Accuracy: 0.9696 | 


In [21]:
model.load("models/step5_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep5,{loss},{acc}")

Duration of evaluation on test data: 78.67 seconds
Test Loss: 0.1029 | Test Accuracy: 0.9763 | 


In [22]:
model.load("models/step6_best.pt")
loss, acc = eval_model(model, test_dataset, criterion, "cuda", num_workers=6)

with open("testing.csv", "a") as f:
    f.write(f"\nstep6,{loss},{acc}")

Duration of evaluation on test data: 76.33 seconds
Test Loss: 0.0330 | Test Accuracy: 0.9923 | 


# Retriever

In [1]:
import os
import dotenv

import pandas as pd

from thesession.retriever import TheSessionRetriever, get_database_url

dotenv.load_dotenv("thesession-db/.env")

url = get_database_url(
    os.getenv("POSTGRES_USER"),
    os.getenv("POSTGRES_PASSWORD"),
    port=15432,
    database=os.getenv("POSTGRES_DB")
)

retriever = TheSessionRetriever(url, "models/step6_best.pt", backend="soundfile")

/home/maxime/.venvs/session/lib64/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/maxime/.venvs/session/lib64/python3.11/site-packages/torch/functional.py:554: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /pytorch/aten/src/ATen/native/TensorShape.cpp:4314.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
retriever("audio_flac/1_cooleys/36371_0.flac", limit=10, duration=60)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,1,36384,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 14\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.932110
1,1,36379,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 9\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.929763
2,1,36371,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 1\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.922451
3,1,36389,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 19\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.910864
4,1,36383,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 13\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.910122
5,1,36394,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 24\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.900701
6,17758,36231,Fiddler Leahy’s,None,https://thesession.org/tunes/17758,reel,1,X: 1\nT: Fiddler Leahy's\nR: reel\nM: 4/4\nL: ...,0.899702
7,485,59924,The Glentaun,None,https://thesession.org/tunes/485,reel,191,X: 2\nT: The Glentaun\nR: reel\nM: 4/4\nL: 1/8...,0.898790
8,1,36373,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 3\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK:...,0.893274
9,1,36386,Cooley’s,None,https://thesession.org/tunes/1,reel,6090,X: 16\nT: Cooley's\nR: reel\nM: 4/4\nL: 1/8\nK...,0.883754


In [18]:
retriever("audio_flac/8_the_banshee/32850_1.flac", limit=10, duration=60)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,8,32850,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 5\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.945547
1,8,32864,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 19\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8...,0.878519
2,8,32858,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 13\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8...,0.852180
3,8,32849,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 4\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.839488
4,24360,46018,The Rabbit And The Lamb,Crisdean MacDonald,https://thesession.org/tunes/24360,jig,7,X: 2\nT: The Rabbit And The Lamb\nR: jig\nM: 6...,0.833975
5,3484,34596,Adele’s Bath,Jacob Fournel,https://thesession.org/tunes/3484,jig,16,X: 1\nT: Adele's Bath\nR: jig\nM: 6/8\nL: 1/8\...,0.830541
6,17942,55451,John Joe Moroney’s Favourite,Dermot Lernihan,https://thesession.org/tunes/17942,reel,6,X: 1\nT: John Joe Moroney's Favourite\nR: reel...,0.819762
7,8,32846,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 1\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.805094
8,8,32854,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 9\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.804305
9,8,32847,The Banshee,James McMahon,https://thesession.org/tunes/8,reel,4120,X: 2\nT: The Banshee\nR: reel\nM: 4/4\nL: 1/8\...,0.804237


In [28]:
retriever("audio_flac/7346_hommage__edmond_parizeau/3724_4.flac", limit=10, duration=60)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,7346,3724,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 4\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.925326
1,7346,3725,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 5\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.899449
2,7346,3721,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 1\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.896984
3,7346,3722,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 2\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.837845
4,7346,3723,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 3\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.827141
5,7937,13807,Jota Da Maia,None,https://thesession.org/tunes/7937,waltz,23,X: 1\nT: Jota Da Maia\nR: waltz\nM: 3/4\nL: 1/...,0.824698
6,7937,13808,Jota Da Maia,None,https://thesession.org/tunes/7937,waltz,23,X: 2\nT: Jota Da Maia\nR: waltz\nM: 3/4\nL: 1/...,0.806999
7,14494,46477,L’Antre,Fred Guichen,https://thesession.org/tunes/14494,waltz,14,X: 2\nT: L'Antre\nR: waltz\nM: 3/4\nL: 1/8\nK:...,0.796969
8,7116,50095,Cuckold Come Out Of The Amrey,None,https://thesession.org/tunes/7116,polka,82,X: 2\nT: Cuckold Come Out Of The Amrey\nR: pol...,0.796321
9,7346,3726,Hommage à Edmond Parizeau,None,https://thesession.org/tunes/7346,reel,114,X: 6\nT: Hommage à Edmond Parizeau\nR: reel\nM...,0.784521


In [39]:
retriever("audio_flac/64_the_maid_behind_the_bar/40526_3.flac", limit=10, duration=60)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,64,40526,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 2\nT: The Maid Behind The Bar\nR: reel\nM: ...,0.972116
1,64,40534,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 10\nT: The Maid Behind The Bar\nR: reel\nM:...,0.958148
2,64,40535,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 11\nT: The Maid Behind The Bar\nR: reel\nM:...,0.937905
3,64,40543,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 19\nT: The Maid Behind The Bar\nR: reel\nM:...,0.935036
4,64,40536,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 12\nT: The Maid Behind The Bar\nR: reel\nM:...,0.914905
5,64,40525,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 1\nT: The Maid Behind The Bar\nR: reel\nM: ...,0.903666
6,64,40532,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 8\nT: The Maid Behind The Bar\nR: reel\nM: ...,0.882514
7,64,40542,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 18\nT: The Maid Behind The Bar\nR: reel\nM:...,0.881782
8,64,40530,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 6\nT: The Maid Behind The Bar\nR: reel\nM: ...,0.865124
9,64,40539,The Maid Behind The Bar,None,https://thesession.org/tunes/64,reel,4308,X: 15\nT: The Maid Behind The Bar\nR: reel\nM:...,0.857024


In [46]:
retriever("test/LS_30071_laird_of_drumblair_v1.MP3", limit=50, duration=60)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,8795,42555,Wind The Bobbins Up,None,https://thesession.org/tunes/8795,waltz,20,X: 3\nT: Wind The Bobbins Up\nR: waltz\nM: 3/4...,0.888479
1,6398,7706,The International Schottische,None,https://thesession.org/tunes/6398,barndance,33,X: 8\nT: The International Schottische\nR: bar...,0.846205
2,23785,14563,John Nugent,Turlough O’Carolan,https://thesession.org/tunes/23785,jig,3,X: 2\nT: John Nugent\nR: jig\nM: 6/8\nL: 1/8\n...,0.842834
3,15100,27134,The Boston Hop,None,https://thesession.org/tunes/15100,polka,4,X: 2\nT: The Boston Hop\nR: polka\nM: 2/4\nL: ...,0.834274
4,9269,25043,Jerry Holland’s 50th Birthday,Howie MacDonald,https://thesession.org/tunes/9269,reel,9,X: 1\nT: Jerry Holland's 50th Birthday\nR: ree...,0.829744
5,14649,9895,Traveller’s Joy,None,https://thesession.org/tunes/14649,polka,35,X: 2\nT: Traveller's Joy\nR: polka\nM: 2/4\nL:...,0.823836
6,22782,60786,John Nugent,Turlough O’Carolan,https://thesession.org/tunes/22782,jig,3,X: 2\nT: John Nugent\nR: jig\nM: 6/8\nL: 1/8\n...,0.820872
7,18929,4627,Wild Colonial Boy,None,https://thesession.org/tunes/18929,waltz,23,X: 1\nT: Wild Colonial Boy\nR: waltz\nM: 3/4\n...,0.790825
8,15100,27133,The Boston Hop,None,https://thesession.org/tunes/15100,polka,4,X: 1\nT: The Boston Hop\nR: polka\nM: 2/4\nL: ...,0.783781
9,235,8711,Sean Bui,None,https://thesession.org/tunes/235,jig,453,X: 6\nT: Sean Bui\nR: jig\nM: 6/8\nL: 1/8\nK: ...,0.778706


In [44]:
retriever("test/LS_30009.MP3", limit=20, duration=60)

/home/maxime/.venvs/session/lib64/python3.11/site-packages/torchaudio/transforms/_transforms.py:580: UserWarning: Argument 'onesided' has been deprecated and has no influence on the behavior of this module.
  warnings.warn(


,TuneID,TuneVersionID,TuneTitle,TuneAuthor,TuneURL,TuneType,Tunebooks,TuneVersion,Similarity
0,21429,1854,Walby’s,None,https://thesession.org/tunes/21429,waltz,4,X: 1\nT: Walby's\nR: waltz\nM: 3/4\nL: 1/8\nK:...,0.930715
1,14649,9894,Traveller’s Joy,None,https://thesession.org/tunes/14649,polka,35,X: 1\nT: Traveller's Joy\nR: polka\nM: 2/4\nL:...,0.865858
2,17799,40570,Gretle In The Garden,None,https://thesession.org/tunes/17799,strathspey,7,X: 2\nT: Gretle In The Garden\nR: strathspey\n...,0.850619
3,22418,16844,Tom Mason’s,None,https://thesession.org/tunes/22418,waltz,5,X: 1\nT: Tom Mason's\nR: waltz\nM: 3/4\nL: 1/8...,0.850461
4,23551,49930,The Tenth Piper,Will Evans,https://thesession.org/tunes/23551,march,6,X: 3\nT: The Tenth Piper\nR: march\nM: 4/4\nL:...,0.850345
5,14505,267,Goose Cove,Brenda Stubbert,https://thesession.org/tunes/14505,march,9,X: 1\nT: Goose Cove\nR: march\nM: 4/4\nL: 1/8\...,0.849212
6,23551,49929,The Tenth Piper,Will Evans,https://thesession.org/tunes/23551,march,6,X: 2\nT: The Tenth Piper\nR: march\nM: 4/4\nL:...,0.847670
7,7970,6032,Gavotte,None,https://thesession.org/tunes/7970,reel,16,X: 1\nT: Gavotte\nR: reel\nM: 4/4\nL: 1/8\nK: ...,0.847567
8,14072,152,Scottish Pentatonic Jig For Paul,None,https://thesession.org/tunes/14072,jig,6,X: 1\nT: Scottish Pentatonic Jig For Paul\nR: ...,0.847420
9,997,2146,The Ash Grove,None,https://thesession.org/tunes/997,waltz,435,X: 1\nT: The Ash Grove\nR: waltz\nM: 3/4\nL: 1...,0.843961
